<a href="https://colab.research.google.com/github/abarankab/tinkoff_notebooks/blob/master/universum_captcha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознаем капчу

Таски:

* Придумать, как это распараллелить
* Узнать, каких букв нет
* Написать архитектуру
* Аугментация
* Шедулер для LR
* Сделать model.save и обернуть в готовый скриптик

Конвенции:

* скаченные файлы — в папке data

In [0]:
import os
arr = ["requests", "csv", "tdqm", "keras", "pickle", "cv2", "glob", "google.colab", "numpy", "PIL", "tensorflow"]
for elem in arr:
  os.system("pip install " + elem)

In [0]:
!mkdir data
!pip install tqdm
!pip install opencv-python

mkdir: cannot create directory ‘data’: File exists


In [0]:
import requests
import csv
from tqdm import trange
from keras.utils import to_categorical
import keras
from keras.preprocessing.image import img_to_array,load_img
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Input, concatenate
from keras.layers import Conv2D, MaxPooling2D
import keras.backend as K
import pickle
import cv2
import glob, os
from google.colab import files

In [0]:
!ls data

0.jpg	  1223.jpg  1447.jpg  1670.jpg	1894.jpg  332.jpg  556.jpg  77.jpg
1000.jpg  1224.jpg  1448.jpg  1671.jpg	1895.jpg  333.jpg  557.jpg  780.jpg
1001.jpg  1225.jpg  1449.jpg  1672.jpg	1896.jpg  334.jpg  558.jpg  781.jpg
1002.jpg  1226.jpg  144.jpg   1673.jpg	1897.jpg  335.jpg  559.jpg  782.jpg
1003.jpg  1227.jpg  1450.jpg  1674.jpg	1898.jpg  336.jpg  55.jpg   783.jpg
1004.jpg  1228.jpg  1451.jpg  1675.jpg	1899.jpg  337.jpg  560.jpg  784.jpg
1005.jpg  1229.jpg  1452.jpg  1676.jpg	189.jpg   338.jpg  561.jpg  785.jpg
1006.jpg  122.jpg   1453.jpg  1677.jpg	18.jpg	  339.jpg  562.jpg  786.jpg
1007.jpg  1230.jpg  1454.jpg  1678.jpg	1900.jpg  33.jpg   563.jpg  787.jpg
1008.jpg  1231.jpg  1455.jpg  1679.jpg	1901.jpg  340.jpg  564.jpg  788.jpg
1009.jpg  1232.jpg  1456.jpg  167.jpg	1902.jpg  341.jpg  565.jpg  789.jpg
100.jpg   1233.jpg  1457.jpg  1680.jpg	1903.jpg  342.jpg  566.jpg  78.jpg
1010.jpg  1234.jpg  1458.jpg  1681.jpg	1904.jpg  343.jpg  567.jpg  790.jpg
1011.jpg  1235.jpg  1459.jpg  1

In [0]:

fname = 'answers.csv'
answers = []
for i in trange(100):
	r = requests.get('https://kslweb1.spb.ctf.su/kweb1/captcha/image.php')
	answer = r.headers['Set-Cookie'].rsplit(' ', 1)[1].split('=')[1] # магия
	answers += [answer]
	with open('data/%d.jpg' % i, 'wb') as w:
		w.write(r.content)

with open(fname, 'w', newline='') as file:
	writer = csv.writer(file)
	writer.writerows(answers)


 35%|███▌      | 35/100 [00:18<00:33,  1.92it/s]


 70%|███████   | 70/100 [00:36<00:15,  1.92it/s]


100%|██████████| 100/100 [00:52<00:00,  1.92it/s]


In [0]:
len(answers)

1000

In [0]:
data=[]
for i in range(100):
  img = load_img('data/%d.jpg' % i)
  arr = img_to_array(img,  data_format=None) 
  data.append(arr)
X = np.stack(data)
X /= 255
X.shape

(100, 60, 100, 3)

In [0]:
s=''.join(answers)
s=set(s)
idx2char = list(s)
char2idx = {}
for i, x in enumerate(s):
  char2idx[x] = i
with open('dict.pkl','wb') as file:
  pickle.dump(idx2char, file)

In [0]:
with open('dict.pkl','rb') as file:
  char2idx = pickle.load(file)  

In [0]:
answers[:10]

['/', '/', '/', '/', '/', '/', '/', '/', '/', '/']

In [0]:
y = []
y = np.array(y)
for i, answer in enumerate(answers):
  raw_array = []
  for c in answer:
    print(c, answer)
    raw_array.append(to_categorical(char2idx[c], num_classes = 30))
  y = np.stack(raw_array)
  print(y.shape)
y = y.reshape(-1,120)
y_test = y[:201]
y_train = y[201:]
X_train = X[201:]
X_test = X[:201]

/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /
(1, 30)
/ /


ValueError: ignored

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(799, 60, 100, 3)
(49, 120)
(201, 60, 100, 3)
(201, 120)


In [0]:
CNN = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(60, 100, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
   # Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
])

x = Input((60, 100, 3))
h = CNN(x)

outputs = [None]*4
for i in range(4):
  outputs[i] = Dense(30, activation='softmax')(h)
  
output = concatenate(outputs)

model = Model(x, output)

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 100, 3)   0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128)          11046080    input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 30)           3870        sequential_1[1][0]               
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 30)           3870        sequential_1[1][0]               
__________________________________________________________________________________________________
dense_5 (D

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['mae', 'acc'])  # metrics=['mae', 'acc'])         metrics=[metrics.mae, metrics.categorical_accuracy])

model.fit(X_train, y_train, batch_size=16, epochs=100, validation_split = 0, validation_data=(X_test, y_test), shuffle = False)
y_pred = model.predict(X)

ValueError: ignored

In [0]:
y_train.shape

(49, 120)

In [0]:
model.fit?

(449, 120)

In [0]:
K.mean?

In [0]:
'''

test_fname = 'test.csv'
test_answers = []
for i in range(200):
	r = requests.get('https://kslweb1.spb.ctf.su/kweb1/captcha/image.php')
	test_answer = r.headers['Set-Cookie'].rsplit(' ', 1)[1].split('=')[1] # магия
	test_answers += [test_answer]
	with open('test_data/%d.jpg' % i, 'wb') as w:
		w.write(r.content)
    
print(test_answers)

with open(test_fname, 'w', newline='') as file:
	writer = csv.writer(file)
	writer.writerows(test_answers)
  
  
test_data=[]
for i in range(200):
  img = load_img('test_data/%d.jpg' % i)
  arr = img_to_array(img,  data_format=None) 
  test_data.append(arr)
tX = np.stack(test_data)
tX /= 255
tX.shape

ts=''.join(test_answers)
ts=set(s)
tidx2char = list(ts)
tchar2idx = {}
for i, x in enumerate(ts):
  tchar2idx[x] = i
with open('tdict.pkl','wb') as file:
  pickle.dump(tchar2idx, file)

with open('tdict.pkl','rb') as file:
  tchar2idx = pickle.load(file)  
  
ty = []
ty = np.array(ty)
for i, answer in enumerate(test_answers):
  word = test_answers[i]
  raw_array = []
  raw_array = np.array(raw_array)
  for c in word:
    raw_array = np.append(raw_array, to_categorical(tchar2idx[c], num_classes = 30))
  ty = np.concatenate((ty, raw_array), axis = 0)
ty = ty.reshape(-1,120)
  
model.evaluate(tX, ty)'''

"\n\ntest_fname = 'test.csv'\ntest_answers = []\nfor i in range(200):\n\tr = requests.get('https://kslweb1.spb.ctf.su/kweb1/captcha/image.php')\n\ttest_answer = r.headers['Set-Cookie'].rsplit(' ', 1)[1].split('=')[1] # магия\n\ttest_answers += [test_answer]\n\twith open('test_data/%d.jpg' % i, 'wb') as w:\n\t\tw.write(r.content)\n    \nprint(test_answers)\n\nwith open(test_fname, 'w', newline='') as file:\n\twriter = csv.writer(file)\n\twriter.writerows(test_answers)\n  \n  \ntest_data=[]\nfor i in range(200):\n  img = load_img('test_data/%d.jpg' % i)\n  arr = img_to_array(img,  data_format=None) \n  test_data.append(arr)\ntX = np.stack(test_data)\ntX /= 255\ntX.shape\n\nts=''.join(test_answers)\nts=set(s)\ntidx2char = list(ts)\ntchar2idx = {}\nfor i, x in enumerate(ts):\n  tchar2idx[x] = i\nwith open('tdict.pkl','wb') as file:\n  pickle.dump(tchar2idx, file)\n\nwith open('tdict.pkl','rb') as file:\n  tchar2idx = pickle.load(file)  \n  \nty = []\nty = np.array(ty)\nfor i, answer in enu

In [0]:
def load(imagename):
  img = img_to_array(load_img(imagename, target_size=(60, 100, 3)))
  return img


X = np.array([load(file) for file in glob.glob("data/*.jpg")])

In [0]:
def model_predict(model, img):
  result = np.split(model.predict(img), 4)
  captcha = ''
  for i in range(4):
    captcha += idx2char[np.argmax(result[i])]
  return captcha

In [0]:
model.save('model.h5')               
model = load_model('model.h5')       

/usr/local/lib/python3.6/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
files.download("dict.pkl")
files.download("model.h5")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 42794, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
def predict(imgname):
  model = load_model('model.h5')
  with open('dict.pkl','rb') as file:
    idx2char = pickle.load(file)
  x = img_to_array(load_img(imgname), data_format = None)
  x = x.reshape((1, 60, 100, 3))
  y = np.split(model.predict(x).reshape((120,)), 4)
  captcha = ''
  for i in range(4):
    captcha += idx2char[np.argmax(y[i])]
  return captcha
predict('data/0.jpg')

'7t74'

In [0]:
counter = 0
for i in range(y.shape[0]):
    imgname = 'data/' + str(i) + '.jpg'
    if predict(imgname) == answers[i]:
        print('Ura ', predict(imgname), answers[i])
        counter += 1
print(counter/y.shape[0])    
        

In [0]:
answers[0]

'7t74'

In [0]:
'''def acc_pred(y, y_pred):
    np_y = K.get
    counter = []
    for i in range(y.shape[0]):
        flag = False
        y_eval = np.split(y[i], 4)
        y_pred_eval = np.split(y_pred[i], 4)
        for i in range(4):
            if y_eval[i][argmax(y_pred[i])] == 1:
                flag = True
            else:
                flag = False
                break
        if flag:
            counter.append(1)
        else:
            counter.append(0)        
    return K.mean(counter)'''